![HPEDEVlogo](Pictures/hpedevlogo-NB.JPG)    ![Dockerlogo](Pictures/docker.png)  

Let's start by connecting to our docker appliance again (Docker in Docker Appliance)

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}

In [3]:
%login 16.31.86.200

[ssh] Login to 16.31.86.200...
[ssh] host=16.31.86.200 hostname=16.31.86.200 other_conf={'user': None, 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7ff064367ac0>}
[ssh] Successfully logged in.


# Configuring nextcloud in a container

Estimated time: 60 minutes.

Based on the work done in the Docker Dojo during a Grenoble Docker Meetup (cf: https://github.com/Enalean/docker-dojo/tree/master/owncloud).

Nextcloud is a web based application providing services such as calendar data or file sharing e.g.
When we want to contain an application such as nextcloud, there are a certain number of aspects to take in account and solve:
  1. installing the application and its dependencies in the container
  2. allow IP configuration for remote access to the application
  3. allow data persistence at each invocation of the container
  4. allow configuration data persistence at each invocation of the container
  
One possibility would be to run the container from an image and launch the various commands in the container (as we've done previously). We could put that in a script and launch it systematically when we instantiate a container from an image, or rebuild a prepared image to be instantiated later. But there is a better way to achieve what we want to do, and this is by using the automation process by Docker with the Dockerfile.

The Dockerfile is a way to describe all the operations required to create an image from an initial basic one and stacking all the operations to build at the end the final image ready to be instantiated and consumed and thrown away.

Let's start our Dockerfile by creating a simple container from a base image and just installing some software components useful for our environment, and build an image from that:


In [4]:
echo 'FROM centos:7' > Dockerfile
echo 'RUN yum install -y httpd' >> Dockerfile
cat Dockerfile

[ssh] host = 16.31.86.200, cwd = /home/jupyter
FROM centos:7
RUN yum install -y httpd


In [5]:
docker build .

[ssh] host = 16.31.86.200, cwd = /home/jupyter
Sending build context to Docker daemon  207.4kB
Step 1/2 : FROM centos:7
 ---> 8652b9f0cb4c
Step 2/2 : RUN yum install -y httpd
 ---> Running in 12c44da59f45
Loaded plugins: fastestmirror, ovl
Determining fastest mirrors
 * base: centos.mirror.fr.planethoster.net
 * extras: centos.mirror.fr.planethoster.net
 * updates: ftp.rezopole.net
Resolving Dependencies
--> Running transaction check
---> Package httpd.x86_64 0:2.4.6-97.el7.centos will be installed
--> Processing Dependency: httpd-tools = 2.4.6-97.el7.centos for package: httpd-2.4.6-97.el7.centos.x86_64
--> Processing Dependency: system-logos >= 7.92.1-1 for package: httpd-2.4.6-97.el7.centos.x86_64
--> Processing Dependency: /etc/mime.types for package: httpd-2.4.6-97.el7.centos.x86_64
--> Processing Dependency: libaprutil-1.so.0()(64bit) for package: httpd-2.4.6-97.el7.centos.x86_64
--> Processing Dependency: libapr-1.so.0()(64bit) for package: httpd-2.4.6-97.el7.centos.x86_64
--> Ru

In [20]:
docker image ls

[ssh] host = 16.31.86.200, cwd = /home/jupyter
REPOSITORY      TAG       IMAGE ID       CREATED          SIZE
<none>          <none>    283093796290   11 minutes ago   375MB
ubuntu          latest    fb52e22af1b0   2 weeks ago      72.8MB
dockerlab1580   latest    314733fd858d   8 weeks ago      1.06GB
dockerlab       latest    9149de89bb43   8 weeks ago      1.06GB
test            latest    2f8bd0730896   8 weeks ago      589MB
ubuntu          <none>    c29284518f49   2 months ago     72.8MB
hello-world     latest    d1165f221234   6 months ago     13.3kB
centos          7         8652b9f0cb4c   10 months ago    204MB


So we can verify that a new CentOS 7 image has been downloaded and based on it a new image has been created (without name nor tag, just an ID) containing httpd installed with its dependencies. 
Check it by instantiating a container based on that image and launching httpd in it:

In [43]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
docker run -d $ImgId /usr/sbin/httpd

[ssh] host = 16.31.86.200, cwd = /home/jupyter
a8ed9b8adac11dca10d94bd13dad6790ad4a5d722cc0a38bcab6fd1b3251dd2d


In [44]:
ps auxww |  grep http

[ssh] host = 16.31.86.200, cwd = /home/jupyter
jupyter  12231  0.0  0.0 113284  1580 ?        Ss   15:47   0:00 bash -c cd /home/jupyter && env PAGER=cat MAIL=/var/mail/jupyter PATH=/usr/local/bin:/usr/bin SHLVL=39 /usr/bin/bash -c ' ps auxww |  grep http  EXIT_CODE=$? echo 3335385.0700081361code: ${EXIT_CODE}3335385.0700081361 echo 3335385.0700081361pwd: $(pwd)3335385.0700081361 echo 3335385.0700081361env: $(cat -v <(env -0))3335385.0700081361 '
jupyter  12242  0.0  0.0 113280  1200 ?        S    15:47   0:00 /usr/bin/bash -c  ps auxww |  grep http  EXIT_CODE=$? echo 3335385.0700081361code: ${EXIT_CODE}3335385.0700081361 echo 3335385.0700081361pwd: $(pwd)3335385.0700081361 echo 3335385.0700081361env: $(cat -v <(env -0))3335385.0700081361 
jupyter  12244  0.0  0.0 112808   948 ?        S    15:47   0:00 grep http


Strange, no httpd daemon seems to be running whereas you had no error message reported. Let's try to understand why. Look at the containers running:

In [45]:
docker container ls

[ssh] host = 16.31.86.200, cwd = /home/jupyter
CONTAINER ID   IMAGE           COMMAND                  CREATED       STATUS       PORTS                                                                              NAMES
4538595c00ea   dockerlab1580   "/bin/sh -c '/usr/sb…"   8 weeks ago   Up 8 weeks   0.0.0.0:14005->22/tcp, :::14005->22/tcp, 0.0.0.0:14055->80/tcp, :::14055->80/tcp   dockerlab1580_dockerlab1580_1


Ok, so you don't have any container running. Look at one which exited then:

In [46]:
docker container ls -a

[ssh] host = 16.31.86.200, cwd = /home/jupyter
CONTAINER ID   IMAGE           COMMAND                  CREATED          STATUS                      PORTS                                                                              NAMES
a8ed9b8adac1   47a95dfca98a    "/usr/sbin/httpd"        17 seconds ago   Exited (0) 16 seconds ago                                                                                      unruffled_lalande
0fd67c183928   283093796290    "/usr/sbin/httpd"        23 minutes ago   Exited (0) 23 minutes ago                                                                                      frosty_jepsen
4538595c00ea   dockerlab1580   "/bin/sh -c '/usr/sb…"   8 weeks ago      Up 8 weeks                  0.0.0.0:14005->22/tcp, :::14005->22/tcp, 0.0.0.0:14055->80/tcp, :::14055->80/tcp   dockerlab1580_dockerlab1580_1


So here it is ! But as seen previously the container exited with a 0 status code, meaning no error. Why ? Well this is because the httpd process is giving back hand to the user immediately after launch, going into background. So Docker thinks that all the tasks that he has to do where done correctly and exit gracefully. Assess that analysis by issuing the following commands:

In [51]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls -a | grep $ImgId | awk '{print $1}'`
docker diff $CtnId
echo ""
docker history $ImgId
echo ""
docker logs $CtnId

[ssh] host = 16.31.86.200, cwd = /home/jupyter
C /var
C /var/log
C /var/log/httpd
A /var/log/httpd/access_log
A /var/log/httpd/error_log

IMAGE          CREATED          CREATED BY                                      SIZE      COMMENT
47a95dfca98a   8 minutes ago    /bin/sh -c #(nop)  CMD ["/bin/sh" "-c" "/usr…   0B        
03d765ec78d4   8 minutes ago    /bin/sh -c #(nop)  MAINTAINER myself@mydomai…   0B        
283093796290   28 minutes ago   /bin/sh -c yum install -y httpd                 171MB     
8652b9f0cb4c   10 months ago    /bin/sh -c #(nop)  CMD ["/bin/bash"]            0B        
<missing>      10 months ago    /bin/sh -c #(nop)  LABEL org.label-schema.sc…   0B        
<missing>      10 months ago    /bin/sh -c #(nop) ADD file:b3ebbe8bd304723d4…   204MB     

AH00558: httpd: Could not reliably determine the server's fully qualified domain name, using 172.17.0.2. Set the 'ServerName' directive globally to suppress this message


So we checked that we can launch the httpd server from inside an instantiated container made from our image. It created the relevant log files in the container, with just a normal warning. We also checked how our image was built. Note that the image built is around 170 MB larger than the base CentOS 7 one (shown by history) and has sensible modifications shown by the diff command.

It's a good start, but now we would like to have our httpd server started automatically with our container creation and staying alive after launch. And have attribution accordingly ;-) So let's make the following modifications to the Dockerfile

In [52]:
echo 'MAINTAINER myself@mydomain.org' >> Dockerfile
echo 'CMD /usr/sbin/apachectl -DFOREGROUND -k start' >> Dockerfile
echo ""
cat Dockerfile

[ssh] host = 16.31.86.200, cwd = /home/jupyter
FROM centos:7
RUN yum install -y httpd
MAINTAINER myself@mydomain.org
CMD /usr/sbin/apachectl -DFOREGROUND -k start
MAINTAINER myself@mydomain.org
CMD /usr/sbin/apachectl -DFOREGROUND -k start


In [53]:
docker build .

[ssh] host = 16.31.86.200, cwd = /home/jupyter
Sending build context to Docker daemon  205.8kB
Step 1/4 : FROM centos:7
 ---> 8652b9f0cb4c
Step 2/4 : RUN yum install -y httpd
 ---> Using cache
 ---> 283093796290
Step 3/4 : MAINTAINER myself@mydomain.org
 ---> Using cache
 ---> 03d765ec78d4
Step 4/4 : CMD /usr/sbin/apachectl -DFOREGROUND -k start
 ---> Using cache
 ---> 47a95dfca98a
Successfully built 47a95dfca98a


You can remark that all the first steps are very quick. This is because Docker caches steps, and will not repeat them unless the Dockerfile changes. You can modify the Dockerfile by putting the `MAINTAINER` command as the second line and re-launch the build. You'll see that in that case Docker invalidates its cache and restarts.
Secondly, we modified the way our Apache Web server is started by forcing it to stay in foreground after launch. That way Docker will continue to keep alive the container.
Now start a container from that image to check the web server is indeed started

In [54]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
docker run -d $ImgId

[ssh] host = 16.31.86.200, cwd = /home/jupyter
c2a3dcf257a01da4d17f41e737e8d649f568f1b9df31a7cfb5592dffd5f23df7


Now check using the same previous commands that the container is indeed running correctly:

In [59]:
docker container ls
echo ""
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
docker diff $CtnId
echo ""
docker history $ImgId
echo ""
docker logs $CtnId
echo ""
ps auxww | grep httpd | grep -v docker

[ssh] host = 16.31.86.200, cwd = /home/jupyter
CONTAINER ID   IMAGE           COMMAND                  CREATED         STATUS         PORTS                                                                              NAMES
c2a3dcf257a0   47a95dfca98a    "/bin/sh -c '/usr/sb…"   4 minutes ago   Up 4 minutes                                                                                      goofy_chatelet
4538595c00ea   dockerlab1580   "/bin/sh -c '/usr/sb…"   8 weeks ago     Up 8 weeks     0.0.0.0:14005->22/tcp, :::14005->22/tcp, 0.0.0.0:14055->80/tcp, :::14055->80/tcp   dockerlab1580_dockerlab1580_1
C /run
C /run/httpd
A /run/httpd/authdigest_shm.8
A /run/httpd/httpd.pid
C /var
C /var/log
C /var/log/httpd
A /var/log/httpd/access_log
A /var/log/httpd/error_log

IMAGE          CREATED          CREATED BY                                      SIZE      COMMENT
47a95dfca98a   16 minutes ago   /bin/sh -c #(nop)  CMD ["/bin/sh" "-c" "/usr…   0B        
03d765ec78d4   16 minutes ago   /bin/sh

So you can see that now your httpd process is running, has produced more differences (a pid file has been created), more log content. It is also running with a UID (48) which is not known by the host, just the container. So now let's try to reach our newly launched webserver:

In [61]:
curl http://localhost

[ssh] host = 16.31.86.200, cwd = /home/jupyter
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (7) Failed connect to localhost:80; Connection refused


abnormal exit code: 7

Of course that doesn't work ;-) You would have been surprised otherwise don't you ? The major aspect here is that the httpd server you launched is run in an isolated environement, the container, that is NOT the host, so its localhost is different from the localhost of the host. Similarly its IP address is different from the one of the host. Proof:

In [73]:
echo ""
echo "*** host ***"
echo ""
/usr/sbin/ip a | grep -A 2 ens192:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
echo ""
echo "*** CTN $CtnId ***"
echo ""
docker exec $CtnId yum install -y net-tools
echo ""
docker exec $CtnId /usr/sbin/ifconfig | grep -A 2 eth0

[ssh] host = 16.31.86.200, cwd = /home/jupyter
*** host ***
2: ens192: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 00:50:56:8e:a2:84 brd ff:ff:ff:ff:ff:ff
    inet 16.31.86.200/22 brd 16.31.87.255 scope global noprefixroute ens192
       valid_lft forever preferred_lft forever
*** CTN c2a3dcf257a0 ***
Loaded plugins: fastestmirror, ovl
Loading mirror speeds from cached hostfile
 * base: centos.mirror.fr.planethoster.net
 * extras: centos.mirror.fr.planethoster.net
 * updates: ftp.rezopole.net
Package net-tools-2.0-0.25.20131004git.el7.x86_64 already installed and latest version
Nothing to do
eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 172.17.0.2  netmask 255.255.0.0  broadcast 172.17.255.255
        ether 02:42:ac:11:00:02  txqueuelen 0  (Ethernet)


Convinced ? So now try *inside* the container to reach the Web server:

In [74]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
echo ""
echo "*** CTN $CtnId ***"
echo ""
docker exec $CtnId curl http://localhost

[ssh] host = 16.31.86.200, cwd = /home/jupyter
*** CTN c2a3dcf257a0 ***
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd"><html><head>
                                 Dload  Upload   Total   Spent    Left  Speed
100  4897  100  4897    0     0   815k      0 --:--:-- --:--:-- --:--:--  956k
<meta http-equiv="content-type" content="text/html; charset=UTF-8">
		<title>Apache HTTP Server Test Page powered by CentOS</title>
		<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">

    <!-- Bootstrap -->
    <link href="/noindex/css/bootstrap.min.css" rel="stylesheet">
    <link rel="stylesheet" href="noindex/css/open-sans.css" type="text/css" />

<style type="text/css"><!--		 

body {
  font-family: "Open Sans", Helvetica, sans-serif;
  font-weight: 100;
  color: #ccc;
  background: rgba(10, 24, 55, 1);
  font-size: 16px;
}

h2, h3, h4 {
  font-weight:

Success !! So you indeed have a running web server, inside a CentOS 7 distribution container. But that's not fully what you want. What you want is to be able to reach it from outside of the container.

In [78]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
echo ""
echo "*** CTN $CtnId ***"
echo ""
CtnIP=`docker exec $CtnId ifconfig | grep -A 2 eth0 | grep inet | awk '{print $2}'`
echo ""
echo "*** using CTN IP $CtnIP ***"
echo ""
curl http://$CtnIP | grep CentOS
HostIP=`/usr/sbin/ip a | grep -A 2 192: | tail -1 | awk '{print $2}' | cut -d/ -f1`
echo ""
echo "*** using host IP $HostIP ***"
echo ""
curl http://$HostIP | grep CentOS

[ssh] host = 16.31.86.200, cwd = /home/jupyter
*** CTN c2a3dcf257a0 ***
*** using CTN IP ***
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
		<title>Apache HTTP Server Test Page powered by CentOS</title>
                                 Dload  Upload   Total   Spent    Left  Speed
100  4897  100  4897    0     0  2564k      0 --:--:-- --:--:-- --:--:-- 4782k
  		<p class="lead">This page is used to test the proper operation of the <a href="http://apache.org">Apache HTTP server</a> after it has been installed. If you can read this page it means that this site is working properly. This server is powered by <a href="http://centos.org">CentOS</a>.</p>
	  				<h2>Promoting Apache and CentOS</h2>
			  		<p>You are free to use the images below on Apache and CentOS Linux powered HTTP servers.  Thanks for using Apache and CentOS!</p>
				  	<p><a href="http://httpd.apache.org/"><img src="images/apache_pb.gif" alt="[ Powered by Apache ]"></a> <a href="http://www.

abnormal exit code: 1


So it works when you use the container IP (because Linux and Docker set the routing up for you automagically, but it doesn't work if you use the IP address of the host, which is really what you want to provide access to the external world. )

By default, the container ports are not exposed outside of the container. So you can't use your host OS to access your isolated webserver. 

You will have to explicitly redirect trafic arriving on port XXXX on the host (which is accessible from outside your environment) to the container port 80 to allow access to the web server running in the container. This will require a change to the way you invoke the container creation. Let's stop the previous container and relaunch it with the right options:

In [79]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
docker stop $CtnId
docker rm $CtnId
docker run -d -p XXXX:80 $ImgId
echo ""
docker container ls
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
echo ""
echo "*** using host IP ***"
echo ""
HostIP=`/usr/sbin/ip a | grep -A 2 192: | tail -1 | awk '{print $2}' | cut -d/ -f1`
curl http://$HostIP | grep CentOS

[ssh] host = 16.31.86.200, cwd = /home/jupyter
c2a3dcf257a0
c2a3dcf257a0
eaafad0cb1da7caa9da0bac788d821f6bf79a6f6d5a18dd8546d185bc90669c8
CONTAINER ID   IMAGE           COMMAND                  CREATED        STATUS                  PORTS                                                                              NAMES
eaafad0cb1da   47a95dfca98a    "/bin/sh -c '/usr/sb…"   1 second ago   Up Less than a second   0.0.0.0:80->80/tcp, :::80->80/tcp                                                  vigorous_hermann
4538595c00ea   dockerlab1580   "/bin/sh -c '/usr/sb…"   8 weeks ago    Up 8 weeks              0.0.0.0:14005->22/tcp, :::14005->22/tcp, 0.0.0.0:14055->80/tcp, :::14055->80/tcp   dockerlab1580_dockerlab1580_1
*** using host IP ***
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
		<title>Apache HTTP Server Test Page powered by CentOS</title>
100  4897  100  4897    0     0

Now that we have redirected the port (the local port XXXX to the container port 80), we can reach our webserver from outside the container using the host IP address. If that machine is reachable from the Internet, you have a Webserver in a container avaible online.

And as this is the case in our environment, please launch a Web browser of your choice and reach that URL http://IPFW:XXXX to check it.


It's now time to add some content to our web server !
Modify again the Dockerfile to add nextcloud to our image:

In [ ]:
echo 'RUN yum install -y tar unzip' >> Dockerfile
echo 'RUN curl https://download.nextcloud.com/server/releases/nextcloud-22.1.1.zip -o /tmp/nextcloud.zip' >> Dockerfile
echo 'RUN cd /var/www/html/ && unzip -q /tmp/nextcloud.zip' >> Dockerfile

Now re-create a new image based on that Dockerfile, purge the previous content before relaunching a new container (please wait till the end of the operation as the download is around 160MB and can take some time)

In [86]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
docker stop $CtnId
docker rm $CtnId
echo ""
docker build .
echo ""
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
echo ""
docker run -d -p XXXX:80 $ImgId
docker container ls

[ssh] host = 16.31.86.200, cwd = /home/jupyter
4621e862f4c3
4621e862f4c3
Sending build context to Docker daemon  209.4kB
Step 1/7 : FROM centos:7
 ---> 8652b9f0cb4c
Step 2/7 : RUN yum install -y httpd
 ---> Using cache
 ---> 78f0bc0a08b8
Step 3/7 : MAINTAINER myself@mydomain.org
 ---> Using cache
 ---> 89bb5e32c231
Step 4/7 : CMD /usr/sbin/apachectl -DFOREGROUND -k start
 ---> Using cache
 ---> 54123dde4923
Step 5/7 : RUN yum install -y tar unzip
 ---> Using cache
 ---> 0f33870d7477
Step 6/7 : RUN curl https://download.nextcloud.com/server/releases/nextcloud-22.1.1.zip -o /tmp/nextcloud.zip
 ---> Using cache
 ---> 0d323f67a0fa
Step 7/7 : RUN cd /var/www/html/ && unzip -q /tmp/nextcloud.zip
 ---> Running in 7eb26d19f794
Removing intermediate container 7eb26d19f794
 ---> 8f733d50e036
Successfully built 8f733d50e036
711f34bcec442cba4621e803b5fcd0be2081cf2ca5a8cb9b77af0bb09cbd575c
CONTAINER ID   IMAGE           COMMAND                  CREATED                  STATUS                  PORTS

Try now to connect to your nextcloud instance. 

![Owncloud failed](Pictures/owncloud_without_dep.png)

1. What happens?
2. What should you do next to solve the issue ? **Discuss with your trainer if you're stuck !**

Hint, you probably need to add the nextcloud dependencies to be able to launch it. Add the following line to your Dockerfile to solve that:

In [ ]:
echo "RUN yum install -y php php-dom php-mbstring php-pdo php-gd" >> Dockerfile

With that you should be able to use owncloud ! (Note that you need to use that version with CentOS 7 for a PHP dependency management) But we're not done yet !!!
If you log on to your owncloud instance, and start customizing it (login/passwd for admin, storage path), you'll have errors first, that we'll fix later on and then if you `Docker stop` and `Docker rm` the container to relaunch it again, of course, none of this customization will be kept as it's not part of your container content.

So we now have to deal with storage management for our Docker container. First we need to solve the error generated when you tried to configure your nextcloud instance. We had rights issues. Use the following command to help solve the issue:

In [ ]:
docker exec b42f9f6f1034 ls -al /var/www/html
docker exec b42f9f6f1034 ps auxww | grep httpd

The principle is that the owner of the httpd process should have the rights on the nextcloud directory to read and store files there. ** So modify your Dockerfile accordingly and retest **.

Now you should be able to customize your owncloud instance and start using it.

By now you have probably remarked that you have to each time deal with IDs for containers and images, which is not that convenient. Let's fix that. Download the owncloud tar file in your directory and modify the ADD line:

In [ ]:
docker build -t nextcloud .

You now have tagged your image and use it by its name:

In [ ]:
docker image ls

It would be great if you could persist the content from one run to another.  Yes, you can ;-) For that, you need to attach a local directory of your host to your container, and point the setup of your owncloud to that directory instead of the one under `/var/www/html/owncloud`.
Create a `/data` directory on your host, mount it in your container under `/data`, and then point your setup ot it:

In [ ]:
mkdir -p /data

In [ ]:
date > /data/myfile.txt

In [ ]:
cat >> Dockerfile << EOF

In [ ]:
```
VOLUME /data
EOF
```

In [ ]:
docker build -t owncloud .

In [ ]:
docker ps

In [ ]:
docker stop 29c8f5ca3d76 

In [ ]:
docker rm 29c8f5ca3d76

In [ ]:
docker run -d -p 80:80 -v /data:/data owncloud:latest

Now reload the owncloud configuration page in your browser, but this time configure the data folder as in the following screen shot:

![Owncloud Setup](Pictures/owncloud.png)

** If you encounter issues you need to adapt your environment so that the apache user is allowed to write on to the /data directory. **

Your current Dockerfile should look like this at that point:

In [ ]:
cat Dockerfile

In [ ]:
```
FROM centos:6
#FROM fedora:latest
RUN yum install -y httpd
MAINTAINER myself@mydomain.org
RUN yum install -y tar bzip2
COPY owncloud-7.0.15.tar.bz2 /var/www/html/
RUN cd /var/www/html/ && tar xvfj owncloud-7.0.15.tar.bz2 && rm -f owncloud-7.0.15.tar.bz2
RUN yum install -y php php-dom php-mbstring php-pdo php-gd
VOLUME /data
RUN chown -R apache:apache /var/www/html/owncloud /data
CMD /usr/sbin/apachectl -DFOREGROUND -k start
EXPOSE 80
```

Move the example text file you created earlier to your ownClould Documents folder so you can see the file and view the file in ownCloud.

In [ ]:
mv /data/myfile.txt /data/bruno/files/Documents

Open the Documents folder in the ownCloud Web UI. Confirm that the myfile.txt example file is present and then view the contents to check that they match what you created earlier.


In [ ]:
docker ps

In [ ]:
docker stop 23f
docker rm 23f

In [ ]:
docker run -d -p XXXX:80 -v /data:/data owncloud:latest

1. At that point you should find again your data on your owncloud instance right ? But what additional pain point do you have ?
2. Knowing that the owncloud configuration data are located under `/var/www/html/owncloud/config/config.php`  try to adapt the Dockerfile to solve that last issue. **Discuss with your trainer if you're stuck !**
Note : there is more than one way to solve this.

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#551199;"></i>&nbsp;&nbsp;Next Steps

# Lab4 : Using Docker Compose

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="4-WKSHP-Using-Docker-Compose.ipynb" target="New" title="Next LAB: Using Docker Compose"><i class="fas fa-chevron-circle-right" style="color:#551199;"></i></a></h2>

</br>
 <a href="1-WKSHP-Intro-to-Containers-techno.ipynb" target="New" title="Back: Introduction to Containers technologies"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="4-WKSHP-Using-Docker-Compose.ipynb" target="New" title="Next:Using Docker Compose"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
